In [1]:
!pip install pyopenssl
!pip install cryptography
!pip install idna

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from OpenSSL import SSL
from cryptography import x509
from cryptography.x509.oid import NameOID
import idna

from socket import socket
from collections import namedtuple

In [3]:
HostInfo = namedtuple(field_names='cert hostname peername', typename='HostInfo')

def get_certificate(hostname, port):
    hostname_idna = idna.encode(hostname)
    sock = socket()

    sock.connect((hostname, port))
    peername = sock.getpeername()
    ctx = SSL.Context(SSL.SSLv23_METHOD) # most compatible
    ctx.check_hostname = False
    ctx.verify_mode = SSL.VERIFY_NONE

    sock_ssl = SSL.Connection(ctx, sock)
    sock_ssl.set_connect_state()
    sock_ssl.set_tlsext_host_name(hostname_idna)
    sock_ssl.do_handshake()
    cert = sock_ssl.get_peer_certificate()
    crypto_cert = cert.to_cryptography()
    sock_ssl.close()
    sock.close()

    return HostInfo(cert=crypto_cert, peername=peername, hostname=hostname)

In [4]:
test_data = get_certificate("santander.co.uk", 443)

In [5]:
cert_pol = test_data.cert.extensions.get_extension_for_class(x509.CertificatePolicies)
cert_pol = str(cert_pol)
print(cert_pol)

<Extension(oid=<ObjectIdentifier(oid=2.5.29.32, name=certificatePolicies)>, critical=False, value=<CertificatePolicies([<PolicyInformation(policy_identifier=<ObjectIdentifier(oid=2.16.840.1.114028.10.1.5, name=Unknown OID)>, policy_qualifiers=['https://www.entrust.net/rpa'])>, <PolicyInformation(policy_identifier=<ObjectIdentifier(oid=2.23.140.1.2.2, name=Unknown OID)>, policy_qualifiers=None)>])>)>


In [6]:
DV = "2.23.140.1.2.1"
OV = "2.23.140.1.2.2"
EV = "2.23.140.1.1"

if DV in cert_pol:
    print("DV Cert")

elif EV in cert_pol:
    print("EV Cert")

elif OV in cert_pol:
    print("OV Cert")

else:
    print("No CertificatePolicies Found")

OV Cert


In [1]:
!pip install check-tls-certs

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python39\python.exe -m pip install --upgrade pip' command.


Using legacy 'setup.py install' for check-tls-certs, since package 'wheel' is not installed.
    Running setup.py install for check-tls-certs: started
    Running setup.py install for check-tls-certs: finished with status 'done'


In [4]:
import check_tls_certs 


In [6]:
check_tls_certs("test.com")

TypeError: 'module' object is not callable